# ChessGPT

In [1]:
import chess
from chess import Move
import chess.svg
import random
import numpy as np
import openai
import json
import matplotlib
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline  

Explanation AI

In [2]:
# config.json
# {
#     "OPENAI_API_KEY": your api key
# }

f = open("./config.json")
data = json.load(f)
f.close()

In [3]:
openai.api_key = data['OPENAI_API_KEY']
model_engine = "gpt-3.5-turbo" 

In [4]:
intro_message = "You are a chess tutor for a beginner player playing as white."

In [5]:
goal_message = "Your goal is to offer give advice to a beginner player during a game. I will give you a move sequence, and you will comment on the state of the game and the latest move in the sequence, and offer possible next moves for white, stating them in UCI representation (e.g. a1a3). Please answer as if you are speaking directly to the player."

In [6]:
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", "content": intro_message},
        {"role": "user", "content": "Hello, ChatGPT!"},
    ],
    max_tokens=100
)

message = response.choices[0]['message']
print("{}: {}".format(message['role'], message['content']))

assistant: Hello! How can I help you today with your chess game?


In [7]:
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", "content": goal_message},
        {"role": "user", "content": "'1. g1f3 g8f6 2. b1c3 b8c6 3. e2e4 e7e5 4. f3e5 f6e4 5. c3e4 f8b4 6. f1c4 c6e5 7. e1g1 e8g8 8. c4f7 f8f7 9. d2d4 d7d5 10. c1f4 c8e6'"},
    ],
    max_tokens=100
)

message = response.choices[0]['message']
print("{}: {}".format(message['role'], message['content']))

assistant: Hello! From what I see, you're playing a pretty balanced game. Your latest move, 10...c8e6, was a good idea to eliminate the threats of the bishop on f4. Now let's talk about some possible next moves for white.

One idea for white could be to 11. d4e5, attacking the knight on f6, and you can respond with 11...Nf6xd5. Another idea could be to 11. Ng1


Chess AI

In [8]:
# EVALUATION
# https://www.chessprogramming.org/Simplified_Evaluation_Function

points = {
    'P': 100,   # Pawn
    'N': 320,   # Knight
    'B': 330,   # Bishop
    'R': 500,   # Rook
    'Q': 900,   # Queen
    'K': 20000  # King
}

# Piece square tables
# top left: a1, top right: a8, bottom left: h1, bottom right: h8
# modified to match square coordinates in chess library
pst = {
    # Pawn
    'P': np.array([[  0,   5,   5,   0,   5,  10,  50,   0],
                   [  0,  10,  -5,   0,   5,  10,  50,   0],
                   [  0,  10, -10,   0,  10,  20,  50,   0],
                   [  0, -20,   0,  20,  25,  30,  50,   0],
                   [  0, -20,   0,  20,  25,  30,  50,   0],
                   [  0,  10, -10,   0,  10,  20,  50,   0],
                   [  0,  10,  -5,   0,   5,  10,  50,   0],
                   [  0,   5,   5,   0,   5,  10,  50,   0]]),
    # Knight
    'N': np.array([[-50, -40, -30, -30, -30, -30, -40, -50],
                   [-40, -20,   5,   0,   5,   0, -20, -40],
                   [-30,   0,  10,  15,  15,  10,   0, -30],
                   [-30,   5,  15,  20,  20,  15,   0, -30],
                   [-30,   5,  15,  20,  20,  15,   0, -30],
                   [-30,   0,  10,  15,  15,  10,   0, -30],
                   [-40, -20,   5,   0,   5,   0, -20, -40],
                   [-50, -40, -30, -30, -30, -30, -40, -50]]),
    # Bishop
    'B': np.array([[-20, -10, -10, -10, -10, -10, -10, -20],
                   [-10,   5,  10,   0,   5,   0,   0, -10],
                   [-10,   0,  10,  10,   5,   5,   0, -10],
                   [-10,   0,  10,  10,  10,  10,   0, -10],
                   [-10,   0,  10,  10,  10,  10,   0, -10],
                   [-10,   0,  10,  10,   5,   5,   0, -10],
                   [-10,   5,  10,   0,   5,   0,   0, -10],
                   [-20, -10, -10, -10, -10, -10, -10, -20]]),
    # Rook
    'R': np.array([[ 0, -5, -5, -5, -5, -5,  5,  0],
                   [ 0,  0,  0,  0,  0,  0, 10,  0],
                   [ 0,  0,  0,  0,  0,  0, 10,  0],
                   [ 5,  0,  0,  0,  0,  0, 10,  0],
                   [ 5,  0,  0,  0,  0,  0, 10,  0],
                   [ 0,  0,  0,  0,  0,  0, 10,  0],
                   [ 0,  0,  0,  0,  0,  0, 10,  0],
                   [ 0, -5, -5, -5, -5, -5,  5,  0]]),
    # Queen
    'Q': np.array([[-20, -10, -10,   0,  -5, -10, -10, -20],
                   [-10,   0,   5,   0,   0,   0,   0, -10],
                   [-10,   5,   5,   5,   5,   5,   0, -10],
                   [ -5,   0,   5,   5,   5,   5,   0,  -5],
                   [ -5,   0,   5,   5,   5,   5,   0,  -5],
                   [-10,   0,   5,   5,   5,   5,   0, -10],
                   [-10,   0,   0,   0,   0,   0,   0, -10],
                   [-20, -10, -10,  -5,  -5, -10, -10, -20]]),
    # King - middle of game
    'K_middle': np.array([[ 20,  20, -10, -20, -30, -30, -30, -30],
                          [ 30,  20, -20, -30, -40, -40, -40, -40],
                          [ 10,   0, -20, -30, -40, -40, -40, -40],
                          [  0,   0, -20, -40, -50, -50, -50, -50],
                          [  0,   0, -20, -40, -50, -50, -50, -50],
                          [ 10,   0, -20, -30, -40, -40, -40, -40],
                          [ 30,  20, -20, -30, -40, -40, -40, -40],
                          [ 20,  20, -10, -20, -30, -30, -30, -30]]),
    # King - endgame
    'K_end': np.array([[-50, -30, -30, -30, -30, -30, -30, -50],
                       [-30, -30, -10, -10, -10, -10, -20, -40],
                       [-30,   0,  20,  30,  30,  20, -10, -30],
                       [-30,   0,  30,  40,  40,  30,   0, -20],
                       [-30,   0,  30,  40,  40,  30,   0, -20],
                       [-30,   0,  20,  30,  30,  20, -10, -30],
                       [-30, -30, -10, -10, -10, -10, -20, -40],
                       [-50, -30, -30, -30, -30, -30, -30, -50]])
 }

# "Additionally we should define where the ending begins. For me it might be either if:
# Both sides have no queens or
# Every side which has a queen has additionally no other pieces or one minorpiece maximum."
# Use to pick which piece square table to use for king
def is_endgame(board):
    fen_str = board.board_fen()
    if fen_str.lower().count('q') == 0:
        # both sides have no queen
        return True
    else:
        # True if side doesn't have a queen
        white_check = not ('Q' in fen_str)
        black_check = not ('q' in fen_str)
        # check if every side that has a queen has additionally no other pieces or one minor piece maximum
        if not white_check and sum(int(c.isupper()) for c in fen_str) <= 3: # includes queen and king
            white_check = True
        if not black_check and sum(int(c.islower()) for c in fen_str) <= 3:
            black_check = True
        return white_check and black_check

def get_board_points(board):
    # Points for pieces on board + bonus points for piece positions (piece square tables)
    points_diff = 0
    for square_num, piece in board.piece_map().items():
        symbol = piece.symbol()
        if symbol.islower():
            square_num = chess.square_mirror(square_num)
            pts_sign = -1
        else:
            pts_sign = 1
        
        square_coords = (chess.square_file(square_num), chess.square_rank(square_num))

        if symbol.upper() == "K":
            if is_endgame(board):
                symbol += "_end"
            else:
                symbol += "_middle"
        
        points_diff += pts_sign * pst[symbol.capitalize()][square_coords]

    return points_diff

In [9]:
# SEARCH
# currently only returns one move with min/max score, can modify to return dictionaries with multiple moves if we want to randomize game a bit

def minimax(board, depth, maximizing_player, moves_list = ()):
    if depth == 0 or len(list(board.legal_moves)) == 0:
        return moves_list, get_board_points(board)
    
    if maximizing_player:
        value = float('-inf')
        final_moves_list = ()

        for move in list(board.legal_moves):
            board_copy = board.copy()
            board_copy.push(move)
            new_moves_list = moves_list + (move.uci(),)

            best_move, move_points = minimax(board_copy, depth - 1, not maximizing_player, new_moves_list)
            if move_points > value:
                final_moves_list = best_move
                value = move_points
            
        return final_moves_list, value
    
    else:
        value = float('inf')
        final_moves_list = ()

        for move in list(board.legal_moves):
            board_copy = board.copy()
            board_copy.push(move)
            new_moves_list = moves_list + (move.uci(),)

            best_move, move_points = minimax(board_copy, depth - 1, not maximizing_player, new_moves_list)
            if move_points < value:
                final_moves_list = best_move
                value = move_points
            
        return final_moves_list, value


def alpha_beta_fail_hard(board, depth, maximizing_player, alpha = float('-inf'), beta = float('inf'), moves_list = ()):
    if depth == 0 or len(list(board.legal_moves)) == 0:
        return moves_list, get_board_points(board)

    if maximizing_player:
        value = float('-inf')
        final_moves_list = ()

        for move in list(board.legal_moves):
            board_copy = board.copy()
            board_copy.push(move)
            new_moves_list = moves_list + (move.uci(),)
            
            best_move, move_points = alpha_beta_fail_hard(board_copy, depth - 1, not maximizing_player, alpha, beta, new_moves_list)
            if move_points > value:
                final_moves_list = best_move
                value = move_points
            
            if value > beta:
                break
            alpha = max(alpha, value)
        
        return final_moves_list, value
    
    else:
        value = float('inf')
        final_moves_list = ()

        for move in list(board.legal_moves):
            board_copy = board.copy()
            board_copy.push(move)
            new_moves_list = moves_list + (move.uci(),)
            
            best_move, move_points = alpha_beta_fail_hard(board_copy, depth - 1, not maximizing_player, alpha, beta, new_moves_list)
            if move_points < value:
                final_moves_list = best_move
                value = move_points
            
            if value < alpha:
                break
            beta = min(beta, value)
        
        return final_moves_list, value

def alpha_beta_fail_soft(board, depth, maximizing_player, alpha = float('-inf'), beta = float('inf'), moves_list = ()):
    if depth == 0 or len(list(board.legal_moves)) == 0:
        return moves_list, get_board_points(board)

    if maximizing_player:
        value = float('-inf')
        final_moves_list = ()

        for move in list(board.legal_moves):
            board_copy = board.copy()
            board_copy.push(move)
            new_moves_list = moves_list + (move.uci(),)
            
            best_move, move_points = alpha_beta_fail_soft(board_copy, depth - 1, not maximizing_player, alpha, beta, new_moves_list)
            if move_points > value:
                final_moves_list = best_move
                value = move_points
            
            alpha = max(alpha, value)
            if value >= beta:
                break
        
        return final_moves_list, value
    
    else:
        value = float('inf')
        final_moves_list = ()

        for move in list(board.legal_moves):
            board_copy = board.copy()
            board_copy.push(move)
            new_moves_list = moves_list + (move.uci(),)
            
            best_move, move_points = alpha_beta_fail_soft(board_copy, depth - 1, not maximizing_player, alpha, beta, new_moves_list)
            if move_points < value:
                final_moves_list = best_move
                value = move_points
            
            beta = min(beta, value)
            if value <= alpha:
                break
        
        return final_moves_list, value

In [16]:
# GAMEPLAY

class RandomPlayer():
    def __init__(self, color = True):
        self.color = color

    def get_move(self, board):
        return random.choice(list(board.legal_moves))

class AIPlayer():
    def __init__(self, color = True, algo = "minimax", depth = 1):
        self.color = color
        self.algo = algo
        self.depth = depth

    def get_move(self, board):
        if self.algo == "AB_hard":
            moves, _ = alpha_beta_fail_hard(board, self.depth, board.turn)
        elif self.algo == "AB_soft":
            moves, _ = alpha_beta_fail_soft(board, self.depth, board.turn)
        else:
            moves, _ = minimax(board, self.depth, board.turn)
        current_move = ""
        for move in moves:
            if (chess.Move.from_uci(str(move)) in board.legal_moves):
                current_move = move
                break
        if current_move == "":
            current_move = moves[0]
            
        return Move.from_uci(current_move)

# could also add human player for testing later
class HumanPlayer():
    def __init__(self, color = True):
        self.color = color

    def get_move(self, board):
        return None

def who(player):
    if player == chess.WHITE:
        return "WHITE"
    elif player == chess.BLACK:
        return "BLACK"
    return None


def get_ChatGPT_response(current_move, is_white, current_board_string):
    system_message = "You will comment on the current state of this chess game described in an ASCII format. Answer as concisely as possible."
    board_state_message = "This is the current chess board state in ASCII format:"

    current_color = "black"
    if is_white:
        current_color = "white"        
        
    comment_message = "Please comment on the quality of move {} for {}".format(current_move, current_color)    
#     comment_message = "Please comment on the quality on this move {} that was just performed by {}".format(current_move, current_color)    
#     comment_message = "Please comment on how the board state is favorable or not for {}".format(current_color)    

    try_again_message = "Are you sure the move {} is invalid for the current board state? Please make sure your output contains the correct assignment of pieces".format(current_move)
    
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": board_state_message},
            {"role": "user", "content": current_board_string},
            {"role": "user", "content": comment_message},
        ],
        max_tokens=100
    )

    message = response.choices[0]['message']
    
    response_string = "{}: {}".format("ChatGPT commentary", message['content'])
    
    if "not possible" in response_string or "illegal" in response_string:
        response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role": "user", "content": try_again_message},
            {"role": "user", "content": current_board_string},

        ],
        max_tokens=100
        )

        message = response.choices[0]['message']
        response_string = "{}: {}".format("ChatGPT commentary", message['content'])

    print(response_string)
    
def play_game_max_moves(player1, player2, board = None, max_moves = 0, per_side = True):
    if board == None:
        board = chess.Board()
    game_moves = []

    if per_side:
        max_moves *= 2
    print("============================ START GAME ============================")

    while max_moves > 0 and not board.is_game_over(claim_draw=True):
        if board.turn == player1.color:
            print("White's turn")
            move = player1.get_move(board)
        else:
            print("Black's turn")

            move = player2.get_move(board)
            
        print("current move: {}".format(str(move)))
        current_move = board.turn
        get_ChatGPT_response(move, current_move, str(board))

        
        # update game variables.
        game_moves.append(move.uci())
        board.push(move)
        max_moves -= 1
        
        print(str(board))

        print("================================================================")

    if not board.is_game_over(claim_draw=True):
        print("Game not over")
    else:
        outcome = board.outcome()
        if outcome == None:
            t = "DRAW"
            w = None
        else:
            t = str(outcome.termination).split('.')[1]
            w = who(outcome.winner)
        print(f"Outcome: {t}\nWinner: {w}\nNumber of moves: {len(game_moves)}")
    print("================================================================")
    print("Moves:\tWHITE\tBLACK\n        -------------")
    for i in range(int(np.ceil(len(game_moves) / 2))):
        next_moves = game_moves[(i*2):(i*2 + 2)]
        if len(next_moves) == 1:
            print(f"{i+1:>6}  {next_moves[0]}")
        else:
            print(f"{i+1:>6}  {next_moves[0]}\t{next_moves[1]}")
            
    return board

def play_game(player1, player2, board = None):
    if board == None:
        board = chess.Board()
    game_moves = []

    while not board.is_game_over(claim_draw=True):
        if board.turn == player1.color:
            move = player1.get_move(board)
        else:
            move = player2.get_move(board)
        game_moves.append(move.uci())
        board.push(move)

    outcome = board.outcome()
    if outcome == None:
        t = "DRAW"
        w = None
    else:
        t = str(outcome.termination).split('.')[1]
        w = who(outcome.winner)
    print(f"Outcome: {t}\nWinner: {w}\nNumber of moves: {len(game_moves)}")
    print("================================================================")
    print("Moves:\tWHITE\tBLACK\n        -------------")
    for i in range(int(np.ceil(len(game_moves) / 2))):
        next_moves = game_moves[(i*2):(i*2 + 2)]
        if len(next_moves) == 1:
            print(f"{i+1:>6}  {next_moves[0]}")
        else:
            print(f"{i+1:>6}  {next_moves[0]}\t{next_moves[1]}")

def play_random_game():
    r1 = RandomPlayer(color = True) # white
    r2 = RandomPlayer(color = False) # black
    play_game(r1, r2)

On starting board: avg time (sec) over 100 runs (depth 1-3) or 5 runs (depth 4)

* depth 1: Minimax: 0.0078, AB Fail Hard: 0.0080, AB Fail Soft: 0.0077
* depth 2: Minimax: 0.1551, AB Fail Hard: 0.0358, AB Fail Soft: 0.0281
* depth 3: Minimax: 3.4503, AB Fail Hard: 0.3359, AB Fail Soft: 0.2769
* depth 4: Minimax: 73.2331, AB Fail Hard: 1.4265, AB Fail Soft: 1.0175

In [17]:
board = play_game_max_moves(AIPlayer(color = True, algo = "AB_hard", depth = 3), # white
                            AIPlayer(color = False, algo = "AB_soft", depth = 3), # black
                            max_moves = 5)

============================ START GAME ============================
White's turn
current move: g1f3
ChatGPT commentary: The move g1f3 for white is a standard opening move that develops the knight and controls the center, so it is generally considered a good move.
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . N . .
P P P P P P P P
R N B Q K B . R
Black's turn
current move: g8f6
ChatGPT commentary: The move g8f6 is a reasonable move for Black, as it develops the knight and prepares to castle kingside. There is no obvious tactical drawback to this move at this point in the game. However, the quality of the move will depend on the overall strategy of Black and the subsequent moves made by both players.
r n b q k b . r
p p p p p p p p
. . . . . n . .
. . . . . . . .
. . . . . . . .
. . . . . N . .
P P P P P P P P
R N B Q K B . R
White's turn
current move: b1c3
ChatGPT commentary: The move b1c3 for White is a good move. It develops a minor piece, 